In [1]:
import pandas as pd
import numpy as np
import matplotlib

In [2]:
#conda install pycountry

In [3]:
import pycountry

In [4]:
productivity = '~/uniwork/ads/ADS-European-Productivity/Datasets/cleaned_Productivity.csv'
employment = '~/uniwork/ads/ADS-European-Productivity/Datasets/employment_growth.csv'

In [5]:
prod_pd = pd.read_csv(productivity)
emp_pd = pd.read_csv(employment, header=0, nrows=None)


In [6]:
#making data look nice so I can make the 2 datasets work with each other
reformattedData = pd.pivot_table(emp_pd, index='geo', columns='TIME_PERIOD', values='OBS_VALUE')
emp_pd_reformatted = reformattedData.reset_index().rename_axis(None, axis=1)

In [7]:
# drops aggregated values 

indicesToDrop = emp_pd_reformatted[((emp_pd_reformatted.geo == 'EA19') | 
                                 (emp_pd_reformatted.geo == 'EU27_2020') | 
                                 (emp_pd_reformatted.geo == 'EA20') |
                                 (emp_pd_reformatted.geo == 'EU28') |
                                   (emp_pd_reformatted.geo == 'EA'))].index
emp_pd_reformatted.drop(indicesToDrop, inplace=True)

In [8]:
#replace NaN value with column mean
emp_pd_no_nan = emp_pd_reformatted.fillna(emp_pd_reformatted.mean())

/tmp/ipykernel_4991/2953700504.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  emp_pd_no_nan = emp_pd_reformatted.fillna(emp_pd_reformatted.mean())


In [9]:
#change country code from 2 letter to 3 letter
emp_pd_3letter = emp_pd_no_nan
for index, row in emp_pd_3letter.iterrows():
    if row.geo == 'EL':
        row.geo = 'GR'
    if row.geo == 'UK':
        row.geo = 'GB'
#     print(index)
    emp_pd_3letter.at[index, 'geo'] = pycountry.countries.get(alpha_2 = row.geo).alpha_3
    
    

0
1
2
3
4
5
6
7
11
12
13
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [10]:
#list of countries in employment dataset
countries = emp_pd_3letter.geo
countries = np.array(countries)

In [11]:
#get rid of countries in productivity dataset that are not in employment growth
for index, row in prod_pd.iterrows():
    if row.LOCATION not in countries:
        prod_pd.drop(index, inplace=True)

In [12]:
#list of countries in productivity dataset
prod_countries = np.array(prod_pd.LOCATION)

In [13]:
#get rid of countries in employment dataset that are not in productivity
for index, row in emp_pd_3letter.iterrows():
    if row.geo not in prod_countries:
        emp_pd_3letter.drop(index, inplace=True)

In [14]:
#drop 2000 to 2004 in productivity dataset
#prod_pd.drop('2000', axis = 1, inplace = True)
prod_pd.drop('2001', axis = 1, inplace = True)
prod_pd.drop('2002', axis = 1, inplace = True)
prod_pd.drop('2003', axis = 1, inplace = True)
prod_pd.drop('2004', axis = 1, inplace = True)

In [15]:
prod_pd.drop('mean', axis = 1, inplace = True)


In [16]:
prod_pd.head()

,LOCATION,2000,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,AUT,55.825179,60.632340,62.218785,63.814049,63.814456,63.363388,64.291488,64.947960,65.645538,66.080679,66.327394,67.267651,67.184974,67.999464,68.309455,68.309868,69.970148,69.634603
2,BEL,63.089170,67.287921,67.637137,68.887444,68.464336,68.020751,69.053627,68.581978,68.923207,69.517696,70.513657,71.604653,71.674769,71.534022,71.681631,72.259271,74.541737,73.607184
3,BGR,14.861829,17.965425,18.618248,19.229785,19.475667,19.717737,20.847786,21.781488,22.497211,22.460713,22.607390,23.296069,23.897609,24.130056,24.784139,25.687153,25.775778,26.559335
5,CHE,61.437020,64.693828,66.315161,67.595243,68.253612,66.723475,68.620315,68.227615,68.494780,69.759903,70.447419,69.982344,70.431631,71.561236,73.212853,73.619376,74.655861,75.909195
9,CZE,25.998047,32.765277,34.781571,36.256014,36.139167,35.303035,36.128328,36.734795,36.886973,37.004724,37.432375,39.390083,39.251558,40.595969,41.161265,42.272668,43.289836,43.487932


In [17]:
emp_pd_3letter.drop(2022,axis = 1, inplace = True)

In [18]:
emp_pd_3letter.head()

,geo,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AUT,1.2,1.8,1.8,1.9,-0.5,0.8,1.6,1.0,0.3,1.0,0.6,1.3,1.6,1.7,1.1,-1.6,2.0
1,BEL,1.4,1.1,1.7,1.8,-0.2,0.6,1.3,0.4,-0.3,0.4,0.9,1.3,1.6,1.5,1.6,0.1,1.9
2,BGR,2.7,3.3,3.2,2.4,-1.7,-3.9,-2.2,-2.5,-0.4,0.4,0.4,0.5,1.8,-0.1,0.3,-2.3,0.2
3,CHE,0.7,2.2,2.6,2.4,0.5,0.3,2.6,1.8,1.2,1.9,1.5,1.4,0.9,1.0,0.7,-0.5,0.6
5,CZE,1.9,1.3,2.1,2.2,-1.8,-1.0,-0.3,0.4,0.3,0.6,1.4,1.6,1.5,1.3,0.2,-1.7,0.4


In [19]:
prod_timeseries = pd.melt(prod_pd, id_vars = ['LOCATION'], var_name = 'YEAR', value_name= 'PRODUCTIVITY')

In [20]:
emp_timeseries = pd.melt(emp_pd_3letter, id_vars = ['geo'], var_name = 'YEAR', value_name= 'EMPLOYMENT')

In [21]:
data_pd = prod_timeseries
data_pd['EMPLOYMENT'] = emp_timeseries['EMPLOYMENT']

In [22]:
data_pd

,LOCATION,YEAR,PRODUCTIVITY,EMPLOYMENT
0,AUT,2000,55.825179,1.2
1,BEL,2000,63.089170,1.4
2,BGR,2000,14.861829,2.7
3,CHE,2000,61.437020,0.7
4,CZE,2000,25.998047,1.9
...,...,...,...,...
517,PRT,2021,40.508787,NaN
518,ROU,2021,37.514761,NaN
519,SVK,2021,48.091287,NaN
520,SVN,2021,47.075129,NaN


In [23]:
data_pd['PRODUCTIVITY'].corr(data_pd['EMPLOYMENT'])

-0.005835359179973838

In [24]:
data_pd.groupby('LOCATION')[['PRODUCTIVITY', 'EMPLOYMENT']].corr()

PRODUCTIVITY  EMPLOYMENT
LOCATION                                       
AUT      PRODUCTIVITY      1.000000   -0.140046
         EMPLOYMENT       -0.140046    1.000000
BEL      PRODUCTIVITY      1.000000    0.157325
         EMPLOYMENT        0.157325    1.000000
BGR      PRODUCTIVITY      1.000000   -0.351455
         EMPLOYMENT       -0.351455    1.000000
CHE      PRODUCTIVITY      1.000000   -0.378044
         EMPLOYMENT       -0.378044    1.000000
CZE      PRODUCTIVITY      1.000000   -0.300400
         EMPLOYMENT       -0.300400    1.000000
DEU      PRODUCTIVITY      1.000000   -0.007790
         EMPLOYMENT       -0.007790    1.000000
DNK      PRODUCTIVITY      1.000000    0.248435
         EMPLOYMENT        0.248435    1.000000
ESP      PRODUCTIVITY      1.000000    0.113942
         EMPLOYMENT        0.113942    1.000000
EST      PRODUCTIVITY      1.000000    0.115207
         EMPLOYMENT        0.115207    1.000000
FIN      PRODUCTIVITY      1.000000   -0.208065
         EMPLOYMENT       -0.208065    1.000000
FRA      PRODUCTIVITY      1.000000    0.040298
         EMPLOYMENT        0.040298    1.000000
GBR      PRODUCTIVITY      1.000000    0.149823
         EMPLOYMENT        0.149823    1.000000
GRC      PRODUCTIVITY      1.000000   -0.229277
         EMPLOYMENT       -0.229277    1.000000
HRV      PRODUCTIVITY      1.000000    0.423361
         EMPLOYMENT        0.423361    1.000000
HUN      PRODUCTIVITY      1.000000   -0.187772
         EMPLOYMENT       -0.187772    1.000000
IRL      PRODUCTIVITY      1.000000   -0.129099
         EMPLOYMENT       -0.129099    1.000000
ISL      PRODUCTIVITY      1.000000   -0.206007
         EMPLOYMENT       -0.206007    1.000000
ITA      PRODUCTIVITY      1.000000    0.448419
         EMPLOYMENT        0.448419    1.000000
LTU      PRODUCTIVITY      1.000000   -0.114206
         EMPLOYMENT       -0.114206    1.000000
LUX      PRODUCTIVITY      1.000000    0.100454
         EMPLOYMENT        0.100454    1.000000
LVA      PRODUCTIVITY      1.000000    0.121624
         EMPLOYMENT        0.121624    1.000000
NLD      PRODUCTIVITY      1.000000   -0.226659
         EMPLOYMENT       -0.226659    1.000000
NOR      PRODUCTIVITY      1.000000    0.221237
         EMPLOYMENT        0.221237    1.000000
POL      PRODUCTIVITY      1.000000    0.291495
         EMPLOYMENT        0.291495    1.000000
PRT      PRODUCTIVITY      1.000000    0.091263
         EMPLOYMENT        0.091263    1.000000
ROU      PRODUCTIVITY      1.000000   -0.028827
         EMPLOYMENT       -0.028827    1.000000
SVK      PRODUCTIVITY      1.000000    0.214578
         EMPLOYMENT        0.214578    1.000000
SVN      PRODUCTIVITY      1.000000   -0.245874
         EMPLOYMENT       -0.245874    1.000000
SWE      PRODUCTIVITY      1.000000    0.011060
         EMPLOYMENT        0.011060    1.000000